In [25]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ProductsCategories").getOrCreate()

Предположила, что начальные данные должны выглядеть примерно так (dataframe для продуктов, dataframe для категорий и dataframe для их связей между собой):

In [26]:
products_df = spark.createDataFrame([(1, "яблоко"), (2, "груша"), (3, "помидор"), (4, "калькулятор"),], ["product_id", "product_name"])
categories_df = spark.createDataFrame([(1, "фрукты"), (2, "овощи"), (3, "молочные продукты")], ["category_id", "category_name"])
prod_cat_df = spark.createDataFrame([(1, 1), (2, 1), (3, 2)], ["product_id", "category_id"])

In [27]:
products_df.show()
categories_df.show()
prod_cat_df.show()

+----------+------------+
|product_id|product_name|
+----------+------------+
|         1|      яблоко|
|         2|       груша|
|         3|     помидор|
|         4| калькулятор|
+----------+------------+

+-----------+-----------------+
|category_id|    category_name|
+-----------+-----------------+
|          1|           фрукты|
|          2|            овощи|
|          3|молочные продукты|
+-----------+-----------------+

+----------+-----------+
|product_id|category_id|
+----------+-----------+
|         1|          1|
|         2|          1|
|         3|          2|
+----------+-----------+



Метод для получения датафрейма с парами «Имя продукта – Имя категории» (учитывая продукты, у которых нет категории)

In [28]:
def get_products_categories(prod_df, cat_df, prod_cat_df):
  joined_df = prod_df.join(prod_cat_df, prod_df.product_id == prod_cat_df.product_id, 'left').join(cat_df, prod_cat_df.category_id == cat_df.category_id, 'left')
  return joined_df.select("product_name", "category_name")

Результат:

In [29]:
get_products_categories(products_df, categories_df, prod_cat_df).show()

+------------+-------------+
|product_name|category_name|
+------------+-------------+
| калькулятор|         null|
|      яблоко|       фрукты|
|       груша|       фрукты|
|     помидор|        овощи|
+------------+-------------+

